In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(col1, col2):
    return sqrt(mean_squared_error(col1, col2))

root = '/opt/home/revoli/eva/Interformer/result'  # normal_affinity, pGNN
####
method = 'spearman'  # pearson, spearman 
predict_method = 'pred_pIC50'
print(f"method:{method}, predicted_method:{predict_method}")
###
# LSD1
df = pd.read_csv(f'{root}/lsd1_project_ensemble.csv')  # n=55
score = df.corr(method, numeric_only=True)['pIC50'][predict_method]
print(f"n={len(df)}, LSD1:", score)
####
# Mpro169
df = pd.read_csv(f'{root}/mPro_covalent_test_ensemble.csv') # n=142
inter_score = []
for i in range(1, 5):
    tmp_df = df[df['Molecule ID'].str.contains(f'{i}type')]
    table = tmp_df.corr(method, numeric_only=True)['pIC50']
    score = table[predict_method]
    #
    inter_score.append(score)
print(f"n={len(df)}, MPro169-test:", np.mean(inter_score))
###
# Mpro-project
df = pd.read_csv(f'{root}/mPro_project_ensemble.csv')  # n=22
score = df.corr(method, numeric_only=True)['pIC50'][predict_method]  # glide-Sp, -0.479
print(f"n={len(df)}, MPro169-project:", score)
######
# kinase
print('=' * 100)
df = pd.read_csv(f'{root}/kinase_test_ensemble.csv') # n=3443, target=45  # kinase_test_ensemble.loss.csv
print(df.corr(numeric_only=True)['pIC50'])
# Interformer
# Filter
if 'pred_pose' in df:
    filter_df = df[df['pred_pose'] > .0]
    filter_data = []
    for t in filter_df['Target'].unique():
        tmp = filter_df[filter_df['Target'] == t]
        filter_data.append([t, tmp['Molecule ID'].unique().tolist()])

methods = ['pred_pIC50', 'gnina', 'vina', 'pGNN', 'normal_interformer']
for predict_method in methods:
    n = 0
    messages = []
    for t, mids in filter_data:
        tmp_df = df[(df['Target'] == t) & (df['Molecule ID'].isin(mids))]
        if predict_method == 'pred_pIC50':
            if 'pred_pose' in tmp_df:
                tmp_df = tmp_df.sort_values('pred_pose', ascending=False)
            pass
        elif predict_method == 'vina':
            tmp_df = tmp_df.sort_values('vina', ascending=True)
        #
        tmp_df = tmp_df.groupby(['Target', 'Molecule ID']).head(1)
        if len(tmp_df) > 30:
            r = tmp_df.corr(method, numeric_only=True)['pIC50'][predict_method]
            n += len(tmp_df)
            messages.append([t, predict_method, r, len(tmp_df)])
    messages.sort(key=lambda x: x[2], reverse=True)
    print(messages)
    all_r = [x[2] for x in messages]
    print(f"[{predict_method}] Kinase, n={n}, target:{len(all_r)}, avg_per_R:", np.mean(all_r))
    print('=' * 100)
####
# tmp_df
# Stats
# df.hist()
# df[df['Target'] == '3d0e']

method:spearman, predicted_method:pred_pIC50
n=55, LSD1: 0.5464025459235792
n=142, MPro169-test: 0.3001995150888525
n=22, MPro169-project: 0.6604469438121116
pose_rank             0.000078
vina                 -0.156337
gnina                 0.310760
pIC50                 1.000000
normal_interformer    0.223582
pGNN                  0.138205
pred_pIC50            0.248507
pred_pose             0.037265
pred_pIC50_var       -0.063006
Name: pIC50, dtype: float64
[['2ayw', 'pred_pIC50', 0.6064929678409722, 42], ['3kl6', 'pred_pIC50', 0.5516440023328948, 64], ['2i78', 'pred_pIC50', 0.4729165888347667, 78], ['2hzi', 'pred_pIC50', 0.3707876456751722, 117], ['3eqh', 'pred_pIC50', 0.35869814747405304, 47], ['3lpb', 'pred_pIC50', 0.3233939153903755, 132], ['2p2i', 'pred_pIC50', 0.3097117353303203, 158], ['3krj', 'pred_pIC50', 0.29958748179444633, 146], ['2of2', 'pred_pIC50', 0.2712207989281127, 148], ['3l5d', 'pred_pIC50', 0.2635467355521228, 109], ['2oj9', 'pred_pIC50', 0.2582171863050231, 95]